In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import torch
from transformers import BertTokenizer, BertModel

if not firebase_admin._apps:  # Jika aplikasi belum diinisialisasi
    cred = credentials.Certificate("/content/drive/MyDrive/dataset/ith-mobile-firebase-adminsdk-zlte2-43c585b00d.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://ith-mobile-default-rtdb.firebaseio.com/'
    })
    print("Firebase berhasil diinisialisasi.")
else:
    print("Firebase sudah diinisialisasi.")

# Kamus kata untuk kategori
prioritas_kamus = {
    'urgensi': ['penting', 'segera', 'kritis', 'darurat', 'mendesak', 'detline', 'besok'],
    'bimbingan_proyek': ['bimbingan', 'proyek', 'pengembangan'],
    'konsultasi': ['diskusi', 'konsultasi', 'masalah', 'solusi'],
    'ajukan_masalah': ['terdapat masalah', 'kendala', 'kesalahan']
}

non_prioritas_kamus = {
    'urgensi': ['tidak mendesak', 'santai', 'kapan-kapan', 'fleksibel'],
    'bimbingan_proyek': ['pelatihan', 'kursus', 'pendalaman'],
    'konsultasi': ['pertanyaan', 'tanya jawab'],
    'ajukan_masalah': ['masalah kecil', 'tidak mendesak', 'bisa diabaikan']
}

# Fungsi untuk menentukan kategori terbaik berdasarkan kecocokan kata
def tentukan_kategori_berdasarkan_kesamaan(teks):
    teks = teks.lower()  # Ubah teks menjadi lowercase untuk pencocokan kata yang tidak sensitif
    prioritas_scores = {}
    non_prioritas_scores = {}

    # Hitung jumlah kecocokan untuk setiap kategori prioritas
    for kategori, kata_kunci in prioritas_kamus.items():
        prioritas_scores[kategori] = sum(kata in teks for kata in kata_kunci)

    # Hitung jumlah kecocokan untuk setiap kategori non-prioritas
    for kategori, kata_kunci in non_prioritas_kamus.items():
        non_prioritas_scores[kategori] = sum(kata in teks for kata in kata_kunci)

    # Tentukan kategori berdasarkan apakah ada kecocokan dengan prioritas atau non-prioritas
    if any(prioritas_scores.values()):  # Jika ada kecocokan di kategori prioritas
        kategori_terbaik = max(prioritas_scores, key=prioritas_scores.get)
        prioritas_status = "Prioritas"
        score_terbaik = prioritas_scores[kategori_terbaik]
    elif any(non_prioritas_scores.values()):  # Jika ada kecocokan di kategori non-prioritas
        kategori_terbaik = max(non_prioritas_scores, key=non_prioritas_scores.get)
        prioritas_status = "Non Prioritas"
        score_terbaik = non_prioritas_scores[kategori_terbaik]
    else:
        kategori_terbaik = "Tidak Teridentifikasi"
        prioritas_status = "Tidak Teridentifikasi"
        score_terbaik = 0

    print(f"Kategori terbaik berdasarkan kecocokan: {kategori_terbaik} (Score: {score_terbaik})")
    return kategori_terbaik, prioritas_status, score_terbaik


# Fungsi untuk memproses data yang belum diproses dan memperbarui nilai "processed"
def proses_data_dari_firebase():
    ref = db.reference('jadwal')  # Pastikan path benar
    docs = ref.get()  # Ambil semua data jadwal

    if docs:
        for key, data in docs.items():
            # Pastikan field 'processed' ada dan bernilai False
            if data.get("processed") == False:
                deskripsi = data.get("deskripsi", "")
                # Lakukan pemrosesan terhadap deskripsi atau data lainnya
                kategori_terbaik, prioritas_status, score = tentukan_kategori_berdasarkan_kesamaan(deskripsi)

                # Menyimpan hasil ke Firebase
                result_data = {
                    "prioritas": prioritas_status,  # Menambahkan prioritas
                    "score": score                 # Menambahkan score
                }

                # Update data di Firebase
                ref.child(key).update(result_data)

                # Ubah field 'processed' menjadi True setelah diproses
                ref.child(key).update({"processed": True})
                print(f"Data diproses dan ditandai sebagai telah diproses: {result_data}")
    else:
        print("Tidak ada data ditemukan di Firebase.")

# Fungsi untuk memastikan 'processed' ada dan tidak mengganggu data yang sudah ada
def pastikan_processed_ada():
    ref = db.reference('jadwal')  # Pastikan path benar
    docs = ref.get()  # Ambil semua data jadwal

    if docs:
        for key, data in docs.items():
            # Cek apakah field "processed" ada
            if "processed" not in data:
                ref.child(key).update({"processed": False})  # Tambahkan field processed jika belum ada
                print(f"Field 'processed' ditambahkan pada {key} dengan nilai False")
    else:
        print("Tidak ada data ditemukan di Firebase.")

# Menjalankan fungsi untuk memastikan 'processed' ada
pastikan_processed_ada()

# Menjalankan fungsi untuk memproses data dari Firebase
proses_data_dari_firebase()


Firebase sudah diinisialisasi.
